<a href="https://colab.research.google.com/github/uakarsh/Traffic-Sign-Recognition/blob/main/German_Traffic_Light_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [3]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

 99% 604M/612M [00:06<00:00, 85.6MB/s]
100% 612M/612M [00:06<00:00, 98.4MB/s]


In [4]:
import zipfile
x = zipfile.ZipFile('/content/gtsrb-german-traffic-sign.zip')
x.extractall('/content/')

In [5]:
import os
os.remove("/content/gtsrb-german-traffic-sign.zip")
print("File Removed!")

File Removed!


In [6]:
import matplotlib.pyplot as plt
import cv2
x = cv2.imread('/content/train/0/00000_00000_00000.png')
from google.colab.patches import cv2_imshow
cv2_imshow(x)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = '/content/train'
train_datagenerator = ImageDataGenerator(rescale=1./255,validation_split=0.15)
train = train_datagenerator.flow_from_directory(train_dir,class_mode = 'categorical',target_size=(30,30))

Found 39209 images belonging to 43 classes.


In [8]:
import tensorflow as tf
import keras

In [9]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,BatchNormalization,Dropout,MaxPooling2D,Activation
chanDim = -1
inputShape = (30,30,3)
classes = 43
epochs = 30
'''Model Making Starts'''
model = Sequential()
model.add(Conv2D(8,(5,5),padding='same',input_shape=inputShape))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size = (2,2)))


# First Set of Convolution
model.add(Conv2D(16,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(16,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2)))

# Second Set of Convolution
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2)))


# First Set of FC Network => RELU Layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Second Set of FC Network => RELU Layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Softmax Classifier
model.add(Dense(classes))
model.add(Activation('softmax'))

In [10]:
opt = keras.optimizers.Adam(lr=0.01,decay = 0.01/(epochs*0.5))
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [11]:
H = model.fit(train,verbose = 1,epochs = epochs)

Epoch 1/30
1226/1226 [==============================] - 52s 17ms/step - loss: 2.6752 - accuracy: 0.2787
Epoch 2/30
1226/1226 [==============================] - 21s 17ms/step - loss: 0.6173 - accuracy: 0.7992
Epoch 3/30
1226/1226 [==============================] - 20s 16ms/step - loss: 0.3099 - accuracy: 0.9014
Epoch 4/30
1226/1226 [==============================] - 20s 17ms/step - loss: 0.2148 - accuracy: 0.9340
Epoch 5/30
1226/1226 [==============================] - 20s 16ms/step - loss: 0.1563 - accuracy: 0.9522
Epoch 6/30
1226/1226 [==============================] - 20s 17ms/step - loss: 0.1166 - accuracy: 0.9630
Epoch 7/30
1226/1226 [==============================] - 20s 16ms/step - loss: 0.1019 - accuracy: 0.9680
Epoch 8/30
1226/1226 [==============================] - 20s 16ms/step - loss: 0.0826 - accuracy: 0.9748
Epoch 9/30
1226/1226 [==============================] - 20s 17ms/step - loss: 0.0750 - accuracy: 0.9772
Epoch 10/30
1226/1226 [==============================] - 20s 17m

In [12]:
model.save("trafficlight.hdf5")